# Compare the best configurations

In [27]:
# import needed libs

%load_ext autoreload

# Auto reloading causes the kernel to reload the libraries we have
%autoreload 2

# usual imports for visualization, etc.
import numpy as np
import matplotlib.pyplot as plt

import datetime

# make it reproducible
np.random.seed(0)

just_plot = False
# just_plot = True

# show plots inline
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
# Some initializations

from envs import SmartVac

max_episode_steps = 100
results_folder = 'res/'
figs_folder = 'figs/'
output_folder = 'output/'

!mkdir output

A subdirectory or file output already exists.


In [29]:
from cycler import cycler
def set_plt_rc():
    font = {'family': 'serif','size': 12}
    
#     SMALL_SIZE = 8
#     MEDIUM_SIZE = 10
#     BIGGER_SIZE = 12

#     plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
#     plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
#     plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
#     plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
#     plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
#     plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
#     plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


#     plt.rc('font', family='serif')
    plt.rc('font', family=font['family'], size=font['size'])
    
    
#     plt.rc('xtick', labelsize='x-small')
#     plt.rc('ytick', labelsize='x-small')
    
    plt.rc('xtick', labelsize='medium')
    plt.rc('ytick', labelsize='medium')
    
    plt.rc('axes', prop_cycle=(cycler(color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728',
                                         '#9467bd', '#8c564b', '#e377c2', '#7f7f7f',
                                         '#bcbd22', '#17becf']) +
                           cycler(linestyle=['-', '--', ':', '-.','-', '--', ':', '-.','-', '--'])))
    plt.rcParams['axes.spines.right'] = False
    plt.rcParams['axes.spines.top'] = False
    
set_plt_rc()

In [30]:
def run_reinforce_agent(alpha):
    AgentClass = ReinforceAgent
    
    params_str = f'alpha_{alpha}_episodes_{episode_count}'

    agent_name = AgentClass.__name__

    mult_avgs = []
    mult_probs1 = []
    mult_probs2 = []
    for i_test in range(num_of_tests):
        print()
        print(i_test + 1, end=' ')

        # Initialize the agent
        agent = AgentClass(alpha=alpha)

        avgs = []
        probs1 = []
        probs2 = []

        episode_rewards = np.zeros(episode_count)
        for i_episode in range(episode_count):
            done = False
            totalReward = 0

            if i_episode >= plot_count and (i_episode % plot_count == 0):
                avg = np.average(episode_rewards[i_episode - plot_count:i_episode])
                avgs.append(avg)

                # deterministic position
                env.x = 0
                env.y = 1
                obs = env.get_obs()
                prob = agent.get_action_vals_for_obs(obs)
                probs1.append(prob)

                # stochastic position
                env.x = 1
                env.y = 1
                obs = env.get_obs()
                prob = agent.get_action_vals_for_obs(obs)
                probs2.append(prob)

                print('#', end='', flush=True)
                if len(avgs) % 100 == 0:
                    print(i_episode)

            obs = env.reset()
            action = agent.start(obs)

            step = 0
            while not done:
                obs, reward, done = env.step(action)
                action = agent.step(obs, reward, done)
                totalReward += reward

                step += 1
                if step > max_episode_steps:
                    done = True

            episode_rewards[i_episode] = totalReward
            agent.update_for_episode()

        mult_avgs.append(avgs)
        mult_probs1.append(probs1)
        mult_probs2.append(probs2)

    mult_avgs = np.array(mult_avgs)
    avgs = np.mean(mult_avgs, axis=0)
    
    results_tmp = f'\n\nAgent: {agent_name} \tAlpha: {alpha}'
    results_tmp += f'\nAverage: \t\t{np.mean(avgs):5.3f}'
    results_tmp += f'\nBest {plot_count} Average: \t{np.max(avgs):5.3f}'
    results_tmp += f'\nLast {plot_count} Average: \t{avgs[-1]:5.3f}'
    print(results_tmp)
    
    return mult_avgs

def run_reinforce_with_baseline_agent(alpha_theta=0.1, alpha_w=0.01):
    AgentClass = ReinforceWithBaselineAgent
    
    params_str = f'alpha_theta_{alpha_theta}_alpha_w_{alpha_w}_episodes_{episode_count}'

    agent_name = AgentClass.__name__

    mult_avgs = []
    mult_probs1 = []
    mult_probs2 = []
    for i_test in range(num_of_tests):
        print()
        print(i_test + 1, end=' ')

        # Initialize the agent
        agent = AgentClass(alpha_w=alpha_w, alpha_theta=alpha_theta)

        avgs = []
        probs1 = []
        probs2 = []

        episode_rewards = np.zeros(episode_count)
        for i_episode in range(episode_count):
            done = False
            totalReward = 0

            if i_episode >= plot_count and (i_episode % plot_count == 0):
                avg = np.average(episode_rewards[i_episode - plot_count:i_episode])
                avgs.append(avg)

                # deterministic position
                env.x = 0
                env.y = 1
                obs = env.get_obs()
                prob = agent.get_action_vals_for_obs(obs)
                probs1.append(prob)

                # stochastic position
                env.x = 1
                env.y = 1
                obs = env.get_obs()
                prob = agent.get_action_vals_for_obs(obs)
                probs2.append(prob)

                print('#', end='', flush=True)
                if len(avgs) % 100 == 0:
                    print(i_episode)

            obs = env.reset()
            action = agent.start(obs)

            step = 0
            while not done:
                obs, reward, done = env.step(action)
                action = agent.step(obs, reward, done)
                totalReward += reward

                step += 1
                if step > max_episode_steps:
                    done = True

            episode_rewards[i_episode] = totalReward
            agent.update_for_episode()

        mult_avgs.append(avgs)
        mult_probs1.append(probs1)
        mult_probs2.append(probs2)

    mult_avgs = np.array(mult_avgs)
    avgs = np.mean(mult_avgs, axis=0)
    
    results_tmp = f'\n\nAgent: {agent_name} \tAlpha_w: {alpha_w}\tAlpha_theta: {alpha_theta}'
    results_tmp += f'\nAverage: \t\t{np.mean(avgs):5.3f}'
    results_tmp += f'\nBest {plot_count} Average: \t{np.max(avgs):5.3f}'
    results_tmp += f'\nLast {plot_count} Average: \t{avgs[-1]:5.3f}'
    print(results_tmp)
    
    return mult_avgs

def run_actor_critic_agent(alpha_theta=0.1, alpha_w=0.01):
    AgentClass = ActorCriticAgent

    params_str = f'alpha_theta_{alpha_theta}_alpha_w_{alpha_w}_episodes_{episode_count}'

    agent_name = AgentClass.__name__

    mult_avgs = []
    mult_probs1 = []
    mult_probs2 = []
    for i_test in range(num_of_tests):
        print()
        print(i_test + 1, end=' ')

        # Initialize the agent
        agent = AgentClass(alpha_w=alpha_w, alpha_theta=alpha_theta)

        avgs = []
        probs1 = []
        probs2 = []

        episode_rewards = np.zeros(episode_count)
        for i_episode in range(episode_count):
            done = False
            totalReward = 0

            if i_episode >= plot_count and (i_episode % plot_count == 0):
                avg = np.average(episode_rewards[i_episode - plot_count:i_episode])
                avgs.append(avg)

                # deterministic position
                env.x = 0
                env.y = 1
                obs = env.get_obs()
                prob = agent.get_action_vals_for_obs(obs)
                probs1.append(prob)

                # stochastic position
                env.x = 1
                env.y = 1
                obs = env.get_obs()
                prob = agent.get_action_vals_for_obs(obs)
                probs2.append(prob)

                print('#', end='', flush=True)
                if len(avgs) % 100 == 0:
                    print(i_episode)

            obs = env.reset()
            action = agent.start(obs)

            step = 0
            while not done:
                obs, reward, done = env.step(action)
                action = agent.step(obs, reward, done)
                totalReward += reward

                step += 1
                if step > max_episode_steps:
                    done = True

            episode_rewards[i_episode] = totalReward
            agent.update_for_episode()

        mult_avgs.append(avgs)
        mult_probs1.append(probs1)
        mult_probs2.append(probs2)

    mult_avgs = np.array(mult_avgs)
    avgs = np.mean(mult_avgs, axis=0)
    
    results_tmp = f'\n\nAgent: {agent_name} \tAlpha_w: {alpha_w}\tAlpha_theta: {alpha_theta}'
    results_tmp += f'\nAverage: \t\t{np.mean(avgs):5.3f}'
    results_tmp += f'\nBest {plot_count} Average: \t{np.max(avgs):5.3f}'
    results_tmp += f'\nLast {plot_count} Average: \t{avgs[-1]:5.3f}'
    print(results_tmp)
    
    return mult_avgs
    
def plot_agent(mult_avgs, agent_name):
    avgs = np.mean(np.array(mult_avgs), axis=0)
    idx = range(len(avgs))
    
    plt.figure(1, figsize=(7,4))
    plt.plot(np.array(idx)*100, avgs, label=agent_name)
#     plt.title(f'Average Return in {episode_count} episodes')
    plt.xlabel(f'Time')
#     plt.ylabel(f'Average Return per {plot_count} episodes')
    plt.ylabel(f'Total Reward on Episode')
    plt.axhline(y=best_performance, linewidth=1, color="g", linestyle='--')

In [ ]:
from agents import ReinforceAgent, ReinforceWithBaselineAgent, ActorCriticAgent

best_performance = 0.63
env = SmartVac()
num_of_tests = 500
episode_count = 10000
plot_count = int(episode_count / 100)

if not just_plot:
    avgs1 = run_reinforce_agent(alpha=0.125)
    avgs2 = run_reinforce_with_baseline_agent(alpha_w=0.5, alpha_theta=0.25)
    avgs3 = run_actor_critic_agent(alpha_w=0.125, alpha_theta=0.5)


1 ###################################################################################################
2 ###################################################################################################
3 ###################################################################################################
4 ###################################################################################################
5 ###################################################################################################
6 ###################################################################################################
7 ###################################################################################################
8 ###################################################################################################
9 ###################################################################################################
10 ##############################################################################

80 ###################################################################################################
81 ###################################################################################################
82 ###################################################################################################
83 ###################################################################################################
84 ###################################################################################################
85 ###################################################################################################
86 ###################################################################################################
87 ###################################################################################################
88 ###################################################################################################
89 ######################################################################

158 ###################################################################################################
159 ###################################################################################################
160 ###################################################################################################
161 ###################################################################################################
162 ###################################################################################################
163 ###################################################################################################
164 ###################################################################################################
165 ###################################################################################################
166 ###################################################################################################
167 ############################################################

236 ###################################################################################################
237 ###################################################################################################
238 ###################################################################################################
239 ###################################################################################################
240 ###################################################################################################
241 ###################################################################################################
242 ###################################################################################################
243 ###################################################################################################
244 ###################################################################################################
245 ############################################################

314 ###################################################################################################
315 ###################################################################################################
316 ###################################################################################################
317 ###################################################################################################
318 ###################################################################################################
319 ###################################################################################################
320 ###################################################################################################
321 ###################################################################################################
322 ###################################################################################################
323 ############################################################

...

95 ###################################################################################################
96 ###################################################################################################
97 ###################################################################################################
98 ###################################################################################################
99 ###################################################################################################
100 ###################################################################################################
101 ###################################################################################################
102 ###################################################################################################
103 ###################################################################################################
104 #################################################################

173 ###################################################################################################
174 ###################################################################################################
175 ###################################################################################################
176 ###################################################################################################
177 ###################################################################################################
178 ###################################################################################################
179 ###################################################################################################
180 ###################################################################################################
181 ###################################################################################################
182 ############################################################

251 ###################################################################################################
252 ###################################################################################################
253 ###################################################################################################
254 ###################################################################################################
255 ###################################################################################################
256 ###################################################################################################
257 ###################################################################################################
258 ###################################################################################################
259 ###################################################################################################
260 ############################################################

329 ###################################################################################################
330 ###################################################################################################
331 ###################################################################################################
332 ###################################################################################################
333 ###################################################################################################
334 ###################################################################################################
335 ###################################################################################################
336 ###################################################################################################
337 ###################################################################################################
338 ############################################################

407 ##########################################################################################

In [ ]:
# Save Results
import pickle

pkl_file_name = f'{output_folder}smart-vac-comp.pkl'
print(pkl_file_name)

if not just_plot:
    res_vars = {
        'avgs1': avgs1,
        'avgs2': avgs2,
        'avgs3': avgs3,
    }

    with open(pkl_file_name, "wb") as f:
        pickle.dump(res_vars, f)

In [ ]:
# Load Results
import pickle

if just_plot:
    res_vars = {}
    with open(pkl_file_name, "rb") as f:
        res_vars = pickle.load(f)

    for var in res_vars:
        vars()[var] = res_vars[var]

In [ ]:
plot_agent(avgs1, 'REINFORCE')
plot_agent(avgs2, 'REINFORCE w/ Baseline')
plot_agent(avgs3, 'Actor-Critic')
plt.legend()
plt.savefig(f'{figs_folder}smart-vac-comp.pdf')
# plt.savefig(f'{figs_folder}smart-vac-comp.png')
plt.show()

In [ ]:
plt.figure(1, figsize=(7,4))
d2 = [np.mean(x, axis=1) for x in [avgs1, avgs2, avgs3]]
plt.boxplot(d2, showfliers=False)
ax = plt.gca()
ax.set_xticklabels(['REINFORCE', 'REINFORCE w/ Baseline', 'Actor-Critic'])
plt.savefig(f'{figs_folder}smart-vac-comp-box.pdf')
# plt.savefig(f'{figs_folder}smart-vac-comp-box.png')
plt.show()

In [ ]:
plt.figure(1, figsize=(7,5))
se = np.std([np.mean(x, axis=1) for x in [avgs1, avgs2, avgs3]], axis=1) / np.sqrt(num_of_tests)
agents = ['REINFORCE', 'REINFORCE w/ Baseline', 'Actor-Critic']
y = [np.mean(x) for x in [avgs1,avgs2,avgs3]]

plt.bar(range(len(agents)), [np.mean(x) for x in [avgs1,avgs2,avgs3]], alpha=0.2, ecolor='black', capsize=10, yerr=se)
# plt.errorbar(range(len(agents)), y, se, linestyle='None', color='r')
plt.xticks(range(len(agents)), agents)
# plt.ylim(bottom=best_performance-0.13)
plt.ylim(bottom=0.52, top=0.62)
plt.savefig(f'{figs_folder}smart-vac-comp-se.pdf')
# plt.savefig(f'{figs_folder}smart-vac-comp-se.png')
plt.show()

In [ ]:
# plt.figure(1, figsize=(7,10))

# plt.subplot(211)
# d2 = [np.mean(x, axis=1) for x in [avgs1, avgs2, avgs3]]
# plt.boxplot(d2, showfliers=False)
# ax = plt.gca()
# ax.set_xticklabels(['REINFORCE', 'REINFORCE w/ Baseline', 'Actor-Critic'])

# ax = plt.subplot(212)
# se = np.std([np.mean(x, axis=1) for x in [avgs1, avgs2, avgs3]], axis=1) / np.sqrt(num_of_tests)
# agents = ['REINFORCE', 'REINFORCE w/ Baseline', 'Actor-Critic']
# y = [np.mean(x) for x in [avgs1,avgs2,avgs3]]
# plt.bar(range(len(agents)), [np.mean(x) for x in [avgs1,avgs2,avgs3]], alpha=0.5, ecolor='black', capsize=10, yerr=se)
# plt.xticks(range(len(agents)), agents)
# plt.ylim(bottom=0.4)

# # plt.savefig(f'{figs_folder}smart-vac-comp-se-box.pdf')
# # plt.savefig(f'{figs_folder}smart-vac-comp-se-box.png')
# plt.show()

In [ ]:
print(np.std([np.mean(x, axis=1) for x in [avgs1, avgs2, avgs3]], axis=1) / np.sqrt(num_of_tests))
print([np.mean(x) for x in [avgs1,avgs2,avgs3]])

# Compare Best Configuration for negative rewards

In [ ]:
from agents import ReinforceAgent, ReinforceWithBaselineAgent, ActorCriticAgent

best_performance = -1.37
env = SmartVac(terminal_rewards=(-1,-3))
num_of_tests = 500
episode_count = 10000
plot_count = int(episode_count / 100)

if not just_plot:
    # REINFORCE doesn't work in this settings
    # avgs1 = run_reinforce_agent(alpha=0.5)
    avgs2 = run_reinforce_with_baseline_agent(alpha_theta=0.25, alpha_w=0.5)
    avgs3 = run_actor_critic_agent(alpha_theta=0.5, alpha_w=0.5)

In [ ]:
# Save Results
import pickle

pkl_file_name = f'{output_folder}smart-vac-neg-comp.pkl'
print(pkl_file_name)

if not just_plot:
    res_vars = {
        'avgs1': avgs1,
        'avgs2': avgs2,
        'avgs3': avgs3,
    }

    with open(pkl_file_name, "wb") as f:
        pickle.dump(res_vars, f)

In [ ]:
# Load Results
import pickle

if just_plot:
    res_vars = {}
    with open(pkl_file_name, "rb") as f:
        res_vars = pickle.load(f)

    for var in res_vars:
        vars()[var] = res_vars[var]

In [ ]:
# REINFORCE doesn't work in this settings
# plot_agent(avgs1, 'REINFORCE')
plot_agent(avgs2, 'REINFORCE w/ Baseline')
plot_agent(avgs3, 'Actor-Critic')
plt.legend()
plt.savefig(f'{figs_folder}smart-vac-neg-comp.pdf')
# plt.savefig(f'{figs_folder}smart-vac-neg-comp.png')
plt.show()

In [ ]:
plt.figure(1, figsize=(7,4))
# d2 = [np.mean(x, axis=1) for x in [avgs1, avgs2, avgs3]]
d2 = [np.mean(x, axis=1) for x in [avgs2, avgs3]]
plt.boxplot(d2, showfliers=False)
ax = plt.gca()
# ax.set_xticklabels(['REINFORCE', 'REINFORCE w/ Baseline', 'Actor-Critic'])
ax.set_xticklabels(['REINFORCE w/ Baseline', 'Actor-Critic'])
plt.savefig(f'{figs_folder}smart-vac-neg-comp-box.pdf')
# plt.savefig(f'{figs_folder}smart-vac-neg-comp-box.png')
plt.show()

In [ ]:
plt.figure(1, figsize=(7,5))
# se = np.std([np.mean(x, axis=1) for x in [avgs1, avgs2, avgs3]], axis=1) / np.sqrt(num_of_tests)
se = np.std([np.mean(x, axis=1) for x in [avgs2, avgs3]], axis=1) / np.sqrt(num_of_tests)
# agents = ['REINFORCE', 'REINFORCE w/ Baseline', 'Actor-Critic']
agents = ['REINFORCE w/ Baseline', 'Actor-Critic']
y = [np.mean(x) for x in [avgs1,avgs2,avgs3]]

# plt.bar(range(len(agents)), [np.mean(x) for x in [avgs1,avgs2,avgs3]], alpha=0.5, ecolor='black', capsize=10, yerr=se)
plt.bar(range(len(agents)), [np.mean(x) for x in [avgs2,avgs3]], alpha=0.2, ecolor='black', capsize=10, yerr=se)
plt.xticks(range(len(agents)), agents)
plt.ylim(top=-1.38, bottom=-1.46)
plt.savefig(f'{figs_folder}smart-vac-neg-comp-se.pdf')
# plt.savefig(f'{figs_folder}smart-vac-neg-comp-se.png')
plt.show()

In [ ]:
print(np.std([np.mean(x, axis=1) for x in [avgs1, avgs2, avgs3]], axis=1) / np.sqrt(num_of_tests))
print([np.mean(x) for x in [avgs1,avgs2,avgs3]])

# Compare Best Configuration with memory and negative rewards

In [ ]:
from agents import ReinforceMemoryAgent as ReinforceAgent
from agents import ReinforceWithBaselineMemoryAgent as ReinforceWithBaselineAgent
from agents import ActorCriticMemoryAgent as ActorCriticAgent

best_performance = 0.74 - 2
# env = SmartVac()
env = SmartVac(terminal_rewards=(-1,-3))
num_of_tests = 1000
episode_count = 10000
plot_count = int(episode_count / 100)

if not just_plot:
    avgs1 = run_reinforce_agent(alpha=2.0)
    avgs2 = run_reinforce_with_baseline_agent(alpha_w=0.125, alpha_theta=0.5)
    avgs3 = run_actor_critic_agent(alpha_w=0.5, alpha_theta=2.0)

In [ ]:
# Save Results
import pickle

pkl_file_name = f'{output_folder}smart-vac-mem-neg-comp.pkl'
print(pkl_file_name)

if not just_plot:
    res_vars = {
        'avgs1': avgs1,
        'avgs2': avgs2,
        'avgs3': avgs3,
    }

    with open(pkl_file_name, "wb") as f:
        pickle.dump(res_vars, f)

In [ ]:
# Load Results
import pickle

if just_plot:
    res_vars = {}
    with open(pkl_file_name, "rb") as f:
        res_vars = pickle.load(f)

    for var in res_vars:
        vars()[var] = res_vars[var]

In [ ]:
plot_agent(avgs1, 'REINFORCE')
plot_agent(avgs2, 'REINFORCE w/ Baseline')
plot_agent(avgs3, 'Actor-Critic')
plt.legend()
plt.savefig(f'{figs_folder}smart-vac-mem-neg-comp.pdf')
# plt.savefig(f'{figs_folder}smart-vac-mem-neg-comp.png')
plt.show()

In [ ]:
plt.figure(1, figsize=(7,4))
d2 = [np.mean(x, axis=1) for x in [avgs1, avgs2, avgs3]]
plt.boxplot(d2, showfliers=False)
ax = plt.gca()
ax.set_xticklabels(['REINFORCE', 'REINFORCE w/ Baseline', 'Actor-Critic'])
plt.savefig(f'{figs_folder}smart-vac-mem-neg-comp-box.pdf')
# plt.savefig(f'{figs_folder}smart-vac-mem-neg-comp-box.png')
plt.show()

In [ ]:
plt.figure(1, figsize=(7,5))
se = np.std([np.mean(x, axis=1) for x in [avgs1, avgs2, avgs3]], axis=1) / np.sqrt(num_of_tests)
agents = ['REINFORCE', 'REINFORCE w/ Baseline', 'Actor-Critic']
y = [np.mean(x) for x in [avgs1,avgs2,avgs3]]

plt.bar(range(len(agents)), [np.mean(x) for x in [avgs1,avgs2,avgs3]], alpha=0.2, ecolor='black', capsize=10, yerr=se)
plt.xticks(range(len(agents)), agents)
# plt.ylim(bottom=best_performance-0.23)
plt.ylim(bottom=-3.2, top=-1.2)
plt.savefig(f'{figs_folder}smart-vac-mem-neg-comp-se.pdf')
# plt.savefig(f'{figs_folder}smart-vac-mem-neg-comp-se.png')
plt.show()

In [ ]:
print(np.std([np.mean(x, axis=1) for x in [avgs1, avgs2, avgs3]], axis=1) / np.sqrt(num_of_tests))
print([np.mean(x) for x in [avgs1,avgs2,avgs3]])